## The Big Wave
Progect Leads: Trevor McLean, Avery Meyers

We will be doing a case study excersise in simplifying and visually representing the progression and creation of three tsunamis through
an analysis of the water level data from several stations. We will be creating a map with the three epicenters and locations of stations 
from which the data was obtained, as well as radius rings from the epicenters of amplitude at distance. As well we have a bar graph of 
the high points of amplitude for all three tsunamis against one another for the first 30 min range after start. And lastly we will have 
graphs of general data about the water level caused by the tsunamis obtained from the stations. (We may have a dedicated graph for
representing amplitude over time for all three tsunamis together to highlight patterns.)

Data of water level from three stations per tsunami will be necessary, as well as general history and general statistics about the tsunamis and their effects. The only resultant data would be a progression of amplitude form epicenter for each tsunami.

The numpy, matplotlib, datetime, and mpl_toolkits.basemap modules will be necessary.

I expect the project will take about 20 man-hours.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import datetime as dt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset, date2index
import math


def read_data(f_name):
    '''(str) -> (list, list)
    
    Takes in a file of primary water levels against time and parses the data into
    two lists: a list of times and a list of primary water level(pwl). Returns those
    lists in a tuple.
    '''
    with open(f_name, 'r') as f_temp:
        f_temp.readline()
        f_temp.readline()
        L_time = []
        L_pwl = []
        
        for i in f_temp:
            L_temp = i.strip().split('\t')
            L_time.append(L_temp[0])
            L_pwl.append(float(L_temp[1]))
                         
    return L_time, L_pwl


def read_lat_lon(name):
    with open(name, 'r') as f:
        f.readline()
        stations = {}
        for l in f:
            temp_l = l.strip().split(',')
            stations[temp_l[0]] = (float(temp_l[1]), float(temp_l[2]))
        return stations
            

                         
def data_point_time(L_time):
    '''(list) -> list
    
    Takes in a list of times L_times in the formal of files from the IOC and reduces it down
    to a series of integers representative of the amount of minutes that have passed at any 
    given point. Returns a list of equal length that contains float quantities of minutes 
    passed.
    
    >>> data_point_time(['2011-03-11 00:05:00', '2011-03-11 01:23:00'])
    [5, 83]
    >>> data_point_time(['2011-03-11 00:00:00', '2011-03-11 01:23:00', '2011-03-11 01:23:00', '2011-03-11 01:23:00'])
    [0, 83, 83, 83]
    '''
    
    L_time_initial = L_time[0]
    format_time = L_time_initial.split()
    format_time_list = format_time[0].split('-') + format_time[1].split(':')
    base_time = dt.datetime(year=int(format_time_list[0]),
                            month=int(format_time_list[1]),
                            day=int(format_time_list[2]), 
                            hour=int(format_time_list[3]), 
                            minute=int(format_time_list[4])
                           )
    
    L_time_minutes = []
    for time in L_time:
        temp_time = time.split()
        temp_time_list = temp_time[0].split('-') + temp_time[1].split(':')
        current_time = dt.datetime(year=int(temp_time_list[0]),
                                    month=int(temp_time_list[1]),
                                    day=int(temp_time_list[2]), 
                                    hour=int(temp_time_list[3]), 
                                    minute=int(temp_time_list[4])
                                   )
        time_difference = (current_time - base_time).total_seconds() / 60
        L_time_minutes.append(time_difference)
        
    return L_time_minutes

def calculate_amplitudes(L_time, L_pwl):
    '''(list, list) -> dict
    
    Takes in a list of times L_time and a list of precision water levels L_pwl and calculates
    a right-side amplitude approximation for every wave of the plot. Returns those amplitudes
    as values paired with a key of the time of the peak for each amplitude in a dictionary.
    '''
    
    start = 0
    end = 0
    amplitudes = {}
    for i in range(1, len(L_pwl)):
        if (len(L_pwl) - 1) == i:
            break
            
        if L_pwl[i] < L_pwl[i+1] and L_pwl[i-1] >= L_pwl[i]:
            start = L_pwl[i]
        elif L_pwl[i] > L_pwl[i+1] and L_pwl[i-1] <= L_pwl[i]:
            end = L_pwl[i]
            peak = i
            
        if start and end:
            amplitudes[L_time[peak]] = 100 * round(((abs(end) - abs(start)) / 2), 4)
            start = 0
            end = 0
            
    return amplitudes

def find_start(time_list, pwl_list):
    '''
    (list, list) -> num

    Takes in a list of times and percision water levels and returns an index on the x axis where 
    the start of a tsunami event can be graphed.

    '''
    running_avg = 0
    last_total = 0
    for i in range(1, len(pwl_list)):
        diff = abs(pwl_list[i] - pwl_list[i-1])
        if i > 30:
            if diff > running_avg*6.3:
                return time_list[i]
            else:
                last_total = last_total + diff
                running_avg = last_total/i
        else: 
            last_total = last_total + diff
            running_avg = last_total/i
    return None

def determine_max(sub_dict):
    '''(dict) -> float
    
    '''
    amps = sub_dict['amps']
    ctr = 0
    for time in amps:
        if amps[time] > ctr:
            ctr = amps[time]
    #if time <= sub_dict['start'] + 30:
            
    return ctr
    
        
    

def find_distance(epilat, epilon, stations):
    
    #In Kilometers
    R = 6373.0

    lat1 = math.radians(epilat)
    lon1 = math.radians(epilon)

    distances = []

    for station in stations:
        lat2 = math.radians(station[0])
        lon2 = math.radians(station[1])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        #Haversine formula
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        distances.append(distance)
    return distances

def find_distance2(epilat, epilon, stations):
    
    #In Kilometers
    R = 6373.0

    lat1 = math.radians(epilat)
    lon1 = math.radians(epilon)

    distances = []

    for station in stations:
        lat2 = math.radians(station[0])
        lon2 = math.radians(station[1])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        #Haversine formula
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        distances.append(distance)
    return distances


#### Info gathered from(primarily) USGS.gov and sms-tsunami-warning.com
#### Data gathered from ioc-sealevelmonitoring.org

Tsunamis are large waves caused primarily by earthquakes or other phenomenons of ocean water shifting such as from coastal land-slides and techtonic shifts. 
The term tsunami was adapted from japanese at around world war 2. 'Tsu' meaning harbor and 'Nami' meaning wave.

##### Japan 2011, Miyagi Prefecture Tsunami: Caused by a 9.0 magnitude undersea earthquake on March 11 
##### Chile 2010, Maule: Caused by a 8.8 magnitude earthquake on February 27 
##### Solomon Islands, Lata 2013: Caused by a 8.0 magnitude earthquake on February 06 

In [ ]:
def plot_variation(info_structure, station_name, color):
    '''(list(lists), list(lists)) -> None
    
    Graphs a representation of the water level change over time for multiple different sets of data. 
    Takes in two lists of lists where each internal list of the list time_lists is the times of data collection
    converted to minutes by the function data_point_time. The list pwl_lists contains lists of precision water level
    associated with the respective index of list of time points. This function returns none and prints all given 
    pairs of time and pwl lists. There must be equal amounts of lists in time_lists and pwl_lists. As well, those
    lists must be of equal length.
    '''
    # FIXME (docstring)
    
    plt.figure()
    times = info_structure[station_name]['times']
    pwls = info_structure[station_name]['pwls']
    plt.plot(times, pwls, color + '--') 
    plt.axvline(find_start(times, pwls))
    plt.title(info_structure['name'] + ', ' + station_name)
    plt.xlabel('Minutes')
    plt.ylabel('Percision Water Level(PWL)')
    
 
def plot_amp_max_per_tsunami(info_structures, L_color_code):
    '''(list, list) -> None
    
    '''
   
    plt.figure()
    plt.xlabel('Station')
    plt.ylabel('Amplitude(Centimeters)')
    ctr = 0
    for tsunami in info_structures:
        max_amps = []
        for station in tsunami['station_names']:
            max_amps.append(tsunami[station]['max_amp'])
        plt.bar(tsunami['station_names'], max_amps, label = tsunami['name'], color = L_color_code[ctr])
        ctr += 1
    plt.legend()
    plt.show()
    
 
def hell(info_structures):
    '''(list) -> None
    
    '''
    
    fig = plt.figure()
    ax = fig.add_axes([0.05, 0.05, 0.9, 0.9])
    m = Basemap(projection = 'merc', llcrnrlat = -80, urcrnrlat = 80, llcrnrlon = -180, urcrnrlon = 180, lat_ts = 20, resolution = 'c', lat_0=20, lon_0=-170)
    #, llcrnrlat = -80, urcrnrlat = 80, llcrnrlon = -180, urcrnrlon = 180, lat_ts = 20, resolution = 'c'
    
    m.drawcoastlines()
    m.drawmapboundary(fill_color = 'aqua')
    #m.fillcontinents(color = 'black')
    
    
    colors1= ['b', 'g', 'r']
    colors2= ['darkblue', 'darkgreen', 'darkred']
    for tsunami in info_structures:
        lats = []
        lons = []
        for station in tsunami['station_names']:
            lons.append(tsunami[station]['lat_lon'][1])
            lats.append(tsunami[station]['lat_lon'][0])
            
        x, y = m(lons, lats)
        m.scatter(x, y, marker='D', color=colors1.pop())
        
        x2, y2 = m(tsunami['tsu_lat_lon'][1], tsunami['tsu_lat_lon'][0])
        m.plot(x2, y2, marker='D', color=colors2.pop(), markersize=10)
    
    plt.show()
    
def plot_amplitude(amps1, name, c):

    t1 = []
    a1 = []

    for time in amps1:
        t1.append(time)
        a1.append(amps1[time])

    plt.figure()
    plt.scatter(t1, a1, color = c, s = 1)
    plt.title(name)
    plt.xlabel('Minutes')
    plt.ylabel('Amplitude in cm')
    

def plot_amps_distances(dictionaries):
    color = ['bo--', 'go--', 'ro--']
    c_i = 0
    plt.figure()
    for dictionary in dictionaries:
        station_names = dictionary['station_names']
        epi_lat_lon = dictionary['tsu_lat_lon']
        name = dictionary['name']
        
        stations_lat_lon = []
        max_amps = []
        for s in station_names:
            max_amps.append(dictionary[s]['max_amp'])
            stations_lat_lon.append(dictionary[s]['lat_lon'])

        distances = find_distance(epi_lat_lon[0], epi_lat_lon[1], stations_lat_lon)
        plt.plot(distances, max_amps, color[c_i])
        c_i = c_i + 1
    plt.title('Max Wave Amplitude vs Distance')
    plt.xlabel('Distance from Epicenter in Kilometers')
    plt.ylabel('Max Amplitude in Centimeters')


def info_structure(station_files, lat_lon_file, station_names, tsunami_name):
    '''(list, str) -> dict
    
    '''
    super_dict = {}
    
    latlon_dict = read_lat_lon(lat_lon_file)
    
    index = 0
    for file in station_files:
        sub_dict = {}
        temp_times, temp_pwls = read_data(file)
        sub_dict['long_times'] = temp_times
        sub_dict['pwls'] = temp_pwls
        sub_dict['times'] = data_point_time(temp_times)
        sub_dict['amps'] = calculate_amplitudes(sub_dict['times'], temp_pwls)
        sub_dict['start'] = find_start(sub_dict['times'], temp_pwls)
        sub_dict['max_amp'] = determine_max(sub_dict)
        sub_dict['lat_lon'] = latlon_dict[station_names[index]]
        super_dict[station_names[index]] = sub_dict
        index += 1
        
    super_dict['tsu_lat_lon'] = latlon_dict[tsunami_name]
    super_dict['station_names'] = station_names
    super_dict['name'] = tsunami_name
    
    return super_dict
        

def main():

    Japan_2011 = info_structure(['Japan(Omaezaki).txt', 'Japan(Honiara).txt', 'Japan(Westport_WA).txt'],
                                'Lat_Lon.txt',
                                ['Omaezaki', 'Honiara', 'Westport WA'],
                                'Japan 2011'
                               )

    Chile_2010 = info_structure(['Chile(Omaezaki).txt', 'Chile(Antofagasta_CL).txt', 'Chile(San_Felix_CL).txt'],
                                'Lat_Lon.txt',
                                ['Omaezaki', 'Antofagasta CL', 'San Felix CL'],
                                'Chile 2010'
                               )

    Solomon_2013 = info_structure(['Solomon_Islands(Honiara).txt', 'Solomon_Islands(Lata_Wharf_SB).txt', 'Solomon_Islands(Lautoka_FJ).txt'],
                                'Lat_Lon.txt',
                                ['Honiara', 'Lata Wharf SB', 'Lautoka FJ'],
                                'Solomon Islands 2013'
                               )
    
    tsunamis = [Japan_2011, Chile_2010, Solomon_2013]

    hell(tsunamis)
    plot_amp_max_per_tsunami(tsunamis, ['b', 'g', 'r'])
    
    colors= ['b', 'g', 'r']
    for i in range(3):
        plot_variation(tsunamis[i], tsunamis[i]['station_names'][0], colors[i])
    for i in range(3):
        plot_variation(tsunamis[i], tsunamis[i]['station_names'][1], colors[i])
    for i in range(3):
        plot_variation(tsunamis[i], tsunamis[i]['station_names'][2], colors[i])
        
    #print(Japan_2011['Omaezaki']['amps'])
    
    plot_amps_distances(tsunamis)
    
    
main()

The above data displays the original behavior of the wave data. Though the points in time that they began is inconsistant, there is still an interesting similarity between each graph as once the event begins, instead of just causing quite high water levels, it instead causes a fluxuation to really high water levels, then to exceptionally low water levels. This as well seems to be following a general trend of shape of the standard fluctuations of water level. This is likely due to the tsunami fluxuations being an alteration of a consistant change in normal tide, meaning that the tide still follows normal behaviour in spite of the tsunami. I may write a function in future to remove the tide change aspect from the graph so that the specific fluctuations of the tsunami can be isolated.